In [184]:
# Import of relevant packages
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# Set random seed 
RSEED = 42

In [185]:
client = pd.read_csv('data/train/client_train.csv')
invoice = pd.read_csv('data/train/invoice_train.csv')

In [186]:
#rename columns
client.rename(columns={
    'disrict': 'district',
}, inplace=True)

invoice.rename(columns={
    'counter_statue': 'counter_status',
    'reading_remarque': 'agent_remark',
    # Add more columns as needed
}, inplace=True)

In [187]:
# Convert to categories for eda viszualizations

# List of columns to convert to categorical
columns_to_convert = ['tarif_type', 'counter_code', 'months_number', 'counter_type']

# Convert each column in invoice
for column in columns_to_convert:
    invoice[column] = invoice[column].astype('category')

In [188]:

# List of columns to convert to categorical
columns_to_convert = ['client_id', 'region','district']

# Convert each column in client
for column in columns_to_convert:
    client[column] = client[column].astype('category')

In [189]:
# convert columns to integer, ensuring support for NaN values
columns_to_convert = ['target']

# Convert each column in client to a pandas nullable integer type
for column in columns_to_convert:
    client[column] = client[column].astype('Int64')

In [191]:
# change strings in counter_status to integers

def convert_to_int(value):
    # Check if the value is 'A' and return 500
    if value == 'A':
        return 500
    # Try to convert numeric strings directly to int
    try:
        return int(value)
    # If conversion fails (which shouldn't happen with the given conditions), return the value
    except ValueError:
        return value
    
invoice['counter_status'] = invoice['counter_status'].apply(convert_to_int)

In [192]:
# convert columns to integer, ensuring support for NaN values
columns_to_convert = ['counter_status']

# Convert each column in client to a pandas nullable integer type
for column in columns_to_convert:
    invoice[column] = invoice[column].astype('Int64')

In [193]:
#Change date to datetime
client['creation_date'] = pd.to_datetime(client['creation_date'])
#Change date to datetime
invoice['invoice_date'] = pd.to_datetime(invoice['invoice_date'])

In [194]:
#Getting unique values on the invoice train data
for col in invoice.columns:
    print(f"{col} - {invoice[col].nunique()}")

client_id - 135493
invoice_date - 8275
tarif_type - 17
counter_number - 201893
counter_status - 12
counter_code - 42
agent_remark - 8
counter_coefficient - 16
consommation_level_1 - 8295
consommation_level_2 - 12576
consommation_level_3 - 2253
consommation_level_4 - 12075
old_index - 155648
new_index - 157980
months_number - 1370
counter_type - 2


In [195]:
#Getting unique values on the invoice train data
for col in client.columns:
    print(f"{col} - {client[col].nunique()}")

district - 4
client_id - 135493
client_catg - 3
region - 25
creation_date - 8088
target - 2


In [196]:
#Get concise information of each column in dataset
invoice.info()
client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4476749 entries, 0 to 4476748
Data columns (total 16 columns):
 #   Column                Dtype         
---  ------                -----         
 0   client_id             object        
 1   invoice_date          datetime64[ns]
 2   tarif_type            category      
 3   counter_number        int64         
 4   counter_status        Int64         
 5   counter_code          category      
 6   agent_remark          int64         
 7   counter_coefficient   int64         
 8   consommation_level_1  int64         
 9   consommation_level_2  int64         
 10  consommation_level_3  int64         
 11  consommation_level_4  int64         
 12  old_index             int64         
 13  new_index             int64         
 14  months_number         category      
 15  counter_type          category      
dtypes: Int64(1), category(4), datetime64[ns](1), int64(9), object(1)
memory usage: 435.5+ MB
<class 'pandas.core.frame.DataFrame'>

In [197]:
# calculate total consumption per billing cycle per counter type

invoice['total_consumption'] = invoice[['consommation_level_1', 'consommation_level_2', 'consommation_level_3', 'consommation_level_4']].sum(axis=1)

In [198]:
# Filter the DataFrame for rows where 'counter_type' is 'ELEC'
invoice_elec = invoice[invoice['counter_type'] == 'ELEC']

# Then, aggregate total_consumption by client_id and counter type (which is now only 'ELEC')
aggregated_consumption_elec = invoice_elec.groupby('client_id')['total_consumption'].agg(
    elec_total_consumption='sum',  # Aggregate the total
    elec_min_consumption='min',
    elec_max_consumption='max',
    elec_std_consumption='std',
    elec_range_consumption=lambda x: x.max() - x.min()  # Calculate the range as max - min
).reset_index()

# Display the aggregated consumption for 'ELEC' counter type
aggregated_consumption_elec

,client_id,elec_total_consumption,elec_min_consumption,elec_max_consumption,elec_std_consumption,elec_range_consumption
0,train_Client_0,12704,38,1386,341.553930,1348
1,train_Client_1,20629,190,1207,197.935960,1017
2,train_Client_10,15057,188,3082,646.808386,2894
3,train_Client_100,24,0,15,3.607011,15
4,train_Client_1000,12917,124,2382,633.485669,2258
...,...,...,...,...,...,...
134810,train_Client_99995,0,0,0,0.000000,0
134811,train_Client_99996,6225,3,831,226.889464,828
134812,train_Client_99997,8910,280,1075,164.270683,795
134813,train_Client_99998,741,206,535,232.638131,329


In [199]:
# Filter the DataFrame for rows where 'counter_type' is 'GAZ'
invoice_gas = invoice[invoice['counter_type'] == 'GAZ']

# Then, aggregate total_consumption by client_id and counter type (which is now only 'ELEC')
aggregated_consumption_gas = invoice_gas.groupby('client_id')['total_consumption'].agg(
    gas_total_consumption='sum',  # Aggregate the total
    gas_min_consumption='min',
    gas_max_consumption='max',
    gas_std_consumption='std',
    gas_range_consumption=lambda x: x.max() - x.min()  # Calculate the range as max - min
).reset_index()

# Display the aggregated consumption for 'ELEC' counter type
aggregated_consumption_gas

,client_id,gas_total_consumption,gas_min_consumption,gas_max_consumption,gas_std_consumption,gas_range_consumption
0,train_Client_10000,4661,0,693,259.190846,693
1,train_Client_100000,1833,0,482,139.262824,482
2,train_Client_100001,4498,0,225,59.176521,225
3,train_Client_100004,516,50,168,44.234602,118
4,train_Client_100005,12738,0,831,261.838010,831
...,...,...,...,...,...,...
62049,train_Client_99992,0,0,0,0.000000,0
62050,train_Client_99993,380,0,83,29.795600,83
62051,train_Client_99995,139,0,139,23.838303,139
62052,train_Client_99996,1426,0,170,58.865869,170


In [200]:
invoice['counter_status'] = invoice['counter_status'].replace(500, 6)

invoice['counter_status'] = invoice['counter_status'].replace(769, 7)

invoice['counter_status'] = invoice['counter_status'].replace(618, 8)

invoice['counter_status'] = invoice['counter_status'].replace(269375, 9)

invoice['counter_status'] = invoice['counter_status'].replace(46, 10)

invoice['counter_status'] = invoice['counter_status'].replace(420, 11)


In [201]:
invoice['agent_remark'] = invoice['agent_remark'].replace(5, 1)
invoice['agent_remark'] = invoice['agent_remark'].replace(6, 2)
invoice['agent_remark'] = invoice['agent_remark'].replace(7, 3)
invoice['agent_remark'] = invoice['agent_remark'].replace(8, 4)
invoice['agent_remark'] = invoice['agent_remark'].replace(9, 5)
invoice['agent_remark'] = invoice['agent_remark'].replace(203, 6)
invoice['agent_remark'] = invoice['agent_remark'].replace(207, 7)
invoice['agent_remark'] = invoice['agent_remark'].replace(413, 8)


In [203]:
#calculate mean counter status
mean_counter_status = invoice.groupby('client_id')['counter_status'].mean().reset_index()


In [204]:
# Calculate mean agent_remark 
mean_agent_remark = invoice.groupby('client_id')['agent_remark'].mean().reset_index()

In [205]:
# Calculate mean counter_coefficient
mean_counter_coefficient = invoice.groupby('client_id')['counter_coefficient'].mean().reset_index()

In [206]:
# Sort invoice DataFrame by 'client_id', 'counter_type', and 'invoice_date'
invoice_sorted = invoice.sort_values(['client_id', 'counter_type', 'invoice_date'])

# Calculate the difference in days between invoice dates within each group of 'client_id' and 'counter_type'
invoice_sorted['invoice_delta_time'] = invoice_sorted.groupby(['client_id', 'counter_type'])['invoice_date'].diff().dt.days

# Create a new DataFrame focusing on the columns of interest
date_eda = invoice_sorted[['client_id', 'counter_type', 'invoice_date', 'invoice_delta_time']].copy()

# Sort this new DataFrame by 'client_id', 'counter_type', and 'invoice_date'
date_eda_sorted = date_eda.sort_values(['client_id', 'counter_type', 'invoice_date'])


In [207]:
# Filter the DataFrame for rows where 'counter_type' is 'ELEC'
date_elec = date_eda_sorted[date_eda_sorted['counter_type'] == 'ELEC']

# Group by both 'client_id' and 'counter_type', then calculate the aggregate statistics for 'invoice_delta_time'
aggregated_elec_date_stats = date_elec.groupby(['client_id'])['invoice_delta_time'].agg(
    elec_min_invoice_delta='min',
    elec_max_invoice_delta='max',
    elec_mean_invoice_delta='mean',
    elec_median_invoice_delta='median',
    elec_std_invoice_delta='std'
).reset_index()


In [208]:
# Filter the DataFrame for rows where 'counter_type' is 'GAZ'
date_gas = date_eda_sorted[date_eda_sorted['counter_type'] == 'GAZ']

# Group by both 'client_id' and 'counter_type', then calculate the aggregate statistics for 'invoice_delta_time'
aggregated_gas_date_stats = date_gas.groupby(['client_id'])['invoice_delta_time'].agg(
    gas_min_invoice_delta='min',
    gas_max_invoice_delta='max',
    gas_mean_invoice_delta='mean',
    gas_median_invoice_delta='median',
    gas_std_invoice_delta='std'
).reset_index()


In [209]:
# List of columns you want to include in the new DataFrame
columns_to_include = ['client_id', 'client_catg', 'region', 'creation_date', 'target']

# Create a new DataFrame with the specified columns
model_df = client[columns_to_include].copy()

In [210]:
model_df = model.merge(mean_counter_status[['client_id', 'counter_status']], on='client_id', how='left')
model_df = model.merge(mean_agent_remark[['client_id', 'agent_remark']], on='client_id', how='left')
model_df = model.merge(mean_counter_coefficient[['client_id', 'counter_coefficient']], on='client_id', how='left')


In [211]:
model_df = model.merge(aggregated_consumption_elec[['client_id', 'elec_total_consumption', 'elec_min_consumption','elec_max_consumption','elec_std_consumption','elec_range_consumption']], on='client_id', how='left')

In [212]:
model_df = model.merge(aggregated_consumption_gas[['client_id', 'gas_total_consumption', 'gas_min_consumption','gas_max_consumption','gas_std_consumption','gas_range_consumption']], on='client_id', how='left')

In [213]:

model_df = model.merge(aggregated_elec_date_stats[['client_id', 'elec_min_invoice_delta', 'elec_max_invoice_delta','elec_mean_invoice_delta','elec_median_invoice_delta','elec_std_invoice_delta']], on='client_id', how='left')


In [214]:

model_df = model.merge(aggregated_gas_date_stats[['client_id', 'gas_min_invoice_delta', 'gas_max_invoice_delta','gas_mean_invoice_delta','gas_median_invoice_delta','gas_std_invoice_delta']], on='client_id', how='left')


In [215]:
model_df

,client_id,client_catg,region,creation_date,target,counter_status,agent_remark,counter_coefficient,elec_total_consumption,elec_min_consumption,...,gas_min_invoice_delta_x,gas_max_invoice_delta_x,gas_mean_invoice_delta_x,gas_median_invoice_delta_x,gas_std_invoice_delta_x,gas_min_invoice_delta_y,gas_max_invoice_delta_y,gas_mean_invoice_delta_y,gas_median_invoice_delta_y,gas_std_invoice_delta_y
0,train_Client_0,11,101,1994-12-31,0,0.000000,6.971429,1.0,12704.0,38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,train_Client_1,11,107,2002-05-29,0,0.000000,7.216216,1.0,20629.0,190.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,train_Client_10,11,301,1986-03-13,0,0.000000,7.055556,1.0,15057.0,188.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,train_Client_100,11,105,1996-07-11,0,0.000000,6.15,1.0,24.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,train_Client_1000,11,303,2014-10-14,0,0.000000,8.857143,1.0,12917.0,124.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135488,train_Client_99995,11,304,2004-07-26,0,0.070423,6.0,1.0,0.0,0.0,...,58.0,250.0,125.393939,121.0,34.681533,58.0,250.0,125.393939,121.0,34.681533
135489,train_Client_99996,11,311,2012-10-25,0,0.268293,8.487805,1.0,6225.0,3.0,...,0.0,231.0,115.350000,120.5,52.220459,0.0,231.0,115.350000,120.5,52.220459
135490,train_Client_99997,11,311,2011-11-22,0,0.000000,9.0,1.0,8910.0,280.0,...,4.0,264.0,132.461538,119.0,78.925720,4.0,264.0,132.461538,119.0,78.925720
135491,train_Client_99998,11,101,1993-12-22,0,0.000000,7.5,1.0,741.0,206.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [216]:


unique_counter_status = invoice['counter_status'].unique()
print(f"Unique counter status: {unique_counter_status}")

unique_counter_codes = np.sort(invoice['counter_code'].unique())
print(f"Unique counter codes: {unique_counter_codes}")

unique_agent_remarks = np.sort(invoice['agent_remark'].unique())
print(f"Unique agent remarks: {unique_agent_remarks}")

unique_counter_coefficients = np.sort(invoice['counter_coefficient'].unique())
print(f"Unique counter coefficients: {unique_counter_coefficients}")

unique_counter_types = np.sort(invoice['counter_type'].unique())
print(f"Unique counter types: {unique_counter_types}")

Unique counter status: <IntegerArray>
[0, 1, 5, 4, 3, 2, 7, 6, 8, 9, 10, 11]
Length: 12, dtype: Int64
Unique counter codes: [  0   1   5  10  16  25  40  65 101 102 201 202 203 204 207 210 214 222
 227 303 305 307 310 317 325 333 367 403 407 410 413 420 433 442 450 453
 467 483 506 532 565 600]
Unique agent remarks: [1 2 3 4 5 6 7 8]
Unique counter coefficients: [ 0  1  2  3  4  5  6  8  9 10 11 20 30 33 40 50]
Unique counter types: ['ELEC' 'GAZ']
